In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import molmodmt as m3t
from simtk import unit

/home/diego/Myopt/Miniconda/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/yank-0.24.0-py3.7-linux-x86_64.egg/yank/experiment.py:1168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """)


In [3]:
system = m3t.load('pdb:1B3T','pdbfixer')

In [4]:
system_fixed = m3t.fix_pdb_structure(system)

In [5]:
m3t.get(system_fixed, n_residues=True, n_aminoacids=True, n_nucleotides=True, n_waters=True, n_ions=True)

[512, 294, 36, 182, 0]

In [6]:
system_fixed_no_solvent = m3t.remove_solvent(system_fixed)

In [7]:
m3t.get(system_fixed_no_solvent, n_residues=True, n_aminoacids=True, n_nucleotides=True, n_waters=True, n_ions=True)

[330, 294, 36, 0, 0]

In [8]:
system_fixed_no_solvent_with_Hs = m3t.add_hydrogens(system_fixed_no_solvent, pH=7.4, verbose=True)

HIS-43 to HIE-43
HIS-68 to HID-68
HIS-144 to HID-144
CYS-173 to CYX-173
HIS-190 to HIE-190
HIS-215 to HID-215
HIS-291 to HID-291
CYS-320 to CYX-320


### With explicit solvent

In [9]:
system_fixed_no_solvent = m3t.convert(system_fixed_no_solvent_with_Hs, 'openmm.Modeller')

In [10]:
m3t.get(system_fixed_no_solvent, net_charge=True)

-24

In [11]:
system_solvated = m3t.solvate(system_fixed_no_solvent, add_hydrogens=True, 
                              box_geometry="truncated_octahedral", clearance=14.0*unit.angstroms)

Hydrogens were removed. The engine building the box will protonate the system.


In [ ]:
fixing_bug = False

if fixing_bug:
    
    for residue in system_solvated.topology.residues():
        if residue.name=='DG5':
            residue.name='DG'
        if residue.name=='DC3':
            residue.name='DC'

    for atom in system_solvated.topology.atoms():
        if atom.name in ["H2'1", "H2'2", "H5'1", "H5'2"]:
            if atom.name=="H2'1":
                atom.name="H2'"
            elif atom.name=="H2'2":
                atom.name="H2''"
            elif atom.name=="H5'1":
                atom.name="H5'"
            elif atom.name=="H5'2":
                atom.name="H5''"

    m3t.convert(system_solvated,'auxfile.pdb')
    system_solvated = m3t.convert('auxfile.pdb', 'openmm.Modeller')
    os.remove('auxfile.pdb')

### Minimization

In [ ]:
system_minimized = m3t.energy_minimization(system_solvated, verbose=True)

In [ ]:
view = m3t.view(system_solvated)
view.clear()
if True:
    view.add_cartoon("protein")
    view.add_cartoon("dna")
else:
    view.add_ball_and_stick("protein")
    view.add_ball_and_stick("dna")
view.add_ball_and_stick("CL-", radiusSize=0.2)
view.add_surface("all", color="cyan", opacity=0.1)
view.center()
view

In [ ]:
m3t.convert(system_minimized, 'system_minimized.pdb')